# Session 2.4.3: Calculating aggregates

## Section 1: Reading the data

As with the other sessions, we can read our data using the Python module _pandas_.

In [1]:
import pandas as pd

In [ ]:
cells = pd.read_csv('https://raw.githubusercontent.com/Flowminder/WB-GDF-Modern-Data-Workflows-Code-Tasks/refs/heads/main/2.4.3_input_data/cells.csv')

In [3]:
cells.head()

,cell_id,cell_type,latitude,longitude,altitude,antenna_height,power_w,range_m,directionality,azimuth_angle,elevation_angle,path_loss_exponent,horizontal_beam_width,vertical_beam_width,azimuth_signal_strength_back_loss,elevation_signal_strength_back_loss
0,c00000,macro,-22.8207,-46.9922,675.824052,25.518755,23.156019,1001.322997,1,259.462219,2.713725,3.2,61.624185,15.544631,21.785002,21.287632
1,c00001,small,-23.8346,-46.9298,671.313542,12.364718,11.154507,829.400040,1,256.045594,4.056964,2.8,93.957550,22.243695,13.830790,15.531173
2,c00002,macro,-23.6522,-46.8299,663.101337,37.379086,57.387554,1977.936637,0,NaN,NaN,3.2,66.665149,8.878208,20.434080,17.183179
3,c00003,small,-23.4802,-46.5457,671.405079,9.487849,14.800546,984.837697,1,13.195942,-7.910196,2.8,97.213181,20.552233,13.645715,14.092863
4,c00004,micro,-23.5195,-46.6204,651.107887,4.715580,2.275979,552.502807,1,109.367828,4.090092,3.0,115.928492,36.064348,11.549988,10.747756


In [ ]:
events = pd.read_parquet('https://github.com/Flowminder/WB-GDF-Modern-Data-Workflows-Code-Tasks/raw/refs/heads/main/2.4.3_input_data/events.parquet')

In [5]:
events.head()

,user_id,cell_id,timestamp
0,932,c00575,2025-03-01 00:00:00
1,76,c01473,2025-03-01 00:00:01
2,217,c01129,2025-03-01 00:00:01
4,879,c01727,2025-03-01 00:00:01
5,840,c03674,2025-03-01 00:00:02


In [ ]:
diaries = pd.read_csv('https://raw.githubusercontent.com/Flowminder/WB-GDF-Modern-Data-Workflows-Code-Tasks/refs/heads/main/2.4.3_input_data/diaries.csv')
diaries['date'] = pd.to_datetime(diaries[['year', 'month', 'day']])
del diaries['year'], diaries['month'], diaries['day']

In [7]:
diaries.head()

,user_id,activity_type,stay_type,longitude,latitude,initial_timestamp,final_timestamp,date
0,0,stay,home,-46.974982,-23.029645,2025-03-01 00:00:00.000000,2025-03-01 23:59:59.000000,2025-03-01
1,1,stay,home,-46.395026,-23.648761,2025-03-01 00:00:00.000000,2025-03-01 10:33:56.577133,2025-03-01
2,1,move,move,NaN,NaN,2025-03-01 10:33:56.577133,2025-03-01 10:46:40.858963,2025-03-01
3,1,stay,other,-46.380671,-23.681435,2025-03-01 10:46:40.858963,2025-03-01 13:59:37.966999,2025-03-01
4,1,move,move,NaN,NaN,2025-03-01 13:59:37.966999,2025-03-01 14:12:15.828044,2025-03-01


## Section 2: Helpful functions

In this exercise, you will compute the average distance travelled from each cell tower. Here, we have written a function to find the distance between two cell towers for you.

In [8]:
import pandas as pd
import numpy as np

# The earth is spherical - we can't just draw straight lines on a plane, 
# so we use the haversine distance to compute the distance between two points on a sphere instead.
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate haversine distance between two points
    Returns distance in meters
    """
    # Convert to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Earth radius in meters
    r = 6_371_000
    
    return c * r

---

## Section 3: Exercises

---

### Task 1: Using the variable `diaries`, count the total number of home stays per cell location per week.

You can use `diaries.date.dt.isocalendar().week` to get the week of the year per date.
This is a small synthetic dataset, but redact anything under 10 people per location per time point.

In [9]:
# Code here:




<details>
<summary><b>🔍 Click to view example code that can solve the task.</b></summary>

<pre><code class="language-python">
diaries.query('stay_type == "home"') \
    .groupby(['longitude', 'latitude', diaries.date.dt.isocalendar().week]) \
    .user_id.count() \
    .reset_index() \
    .query('user_id >= 10').head(40)
</code></pre>

</details>

### Question 1: How many people have a home stay, at cell with coordinates (-47.195806,	-22.959655) on the 10th week of the year?

In [10]:
# Use this space to analyse your aggregate:



# Once you have the answer, add the answer to your multiple choice quiz.

Hint 1: You might need to query the dataframe after you create it, in order to find the correct row.

Hint 2: Make sure to redact the dataset.

Hint 3: The answer is on row 31, if you used `.groupby(['longitude', 'latitude', diaries.date.dt.isocalendar().week])` when aggregating.

---

### Task 2: Using `events` and the function `haversine`, compute the distance between successive events per subscriber. Store this as a column in the events dataframe as a new variable, `events_with_coords`. Remove any consecutive events that happened at the same tower.

In [11]:
# Code here:




<details>
<summary><b>🔍 Click to view example code that can solve the task.</b></summary>

<pre><code class="language-python">
# Step 1: Sort events by user_id and timestamp
events_sorted = events.sort_values(['user_id', 'timestamp'])

# Step 2: Merge with cell tower locations (assuming you have a cells dataframe)
# cells should have columns: cell_id, latitude, longitude
events_with_coords = events_sorted.merge(
    cells[['cell_id', 'latitude', 'longitude']], 
    on='cell_id', 
    how='left'
)

# Step 3: Get previous event's coordinates for each user
events_with_coords['lat_prev'] = events_with_coords.groupby('user_id')['latitude'].shift(1)
events_with_coords['lon_prev'] = events_with_coords.groupby('user_id')['longitude'].shift(1)

# Step 4: Calculate distance between successive events
events_with_coords['distance_m'] = haversine(
    events_with_coords['lon_prev'],
    events_with_coords['lat_prev'],
    events_with_coords['longitude'],
    events_with_coords['latitude']
)

events_with_coords = events_with_coords[events_with_coords.distance_m > 0]

# View results (first event per user will have NaN distance)
print(events_with_coords[['user_id', 'timestamp', 'cell_id', 'distance_m']].head(20))
</code></pre>

</details>

### Question 2: Is what you have produced in Task 2 an aggregate?

Hint: Think about the definition and objectives of an aggregate, from the session slides.

In [12]:
# You don't need to write any code in this box.
# Add the answer to your multiple choice quiz.

---

### Task 3: Compute the average distance travelled from each cell.

Aggregate your previous variable, `events_with_coords` to find the *median* distance travelled from each cell.

If you are struggling, use the code provided in the drop down boxes.

In [13]:
# Code here:




<details>
<summary><b>🔍 Click to view example code that can solve the task.</b></summary>

<pre><code class="language-python">

events_with_coords.groupby('cell_id').distance_m.median()

</code></pre>

</details>

### Question 3: Which cell do people travel the furthest on average from?

In [14]:
# Perform analysis here:



# Check the correct answer on your multiple choice quiz.